In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import glob
import seisbench.models as sbm
from obspy import read, read_events
from obspy import UTCDateTime as utc
import sys
import pickle
from datetime import datetime as dt
import logging
import pandas as pd

In [ ]:
from obspy.signal.trigger import recursive_sta_lta, trigger_onset
from seisbench.util.annotations import Pick, PickList, ClassifyOutput
class stalta:
    def __init__(self):
        self.name = 'STALTA'
        self.sta = 1
        self.lta = 20
        self.trigger_on = 4
        self.trigger_off = 0.5
    def classify(self, stream):
        output = ClassifyOutput('STALTA')
        picks = PickList()
        st = stream.select(channel='*Z')
        for tr in st:
            sps = tr.stats.sampling_rate
            sta_sample = int(self.sta * sps)
            lta_sample = int(self.lta * sps)
            cft = recursive_sta_lta(tr.data, sta_sample, lta_sample)
            # plot_trigger(tr, cft, trigger_on, trigger_of)
            lst_on_of = trigger_onset(cft, self.trigger_on, self.trigger_off)
            for on_of in lst_on_of:
                pick_time = start_time =  tr.stats.starttime + (on_of[0]/sps)
                end_time = tr.stats.starttime + (on_of[1]/sps)
                pick = Pick(trace_id='.'.join(tr.id.split('.')[:-1]),
                            start_time=start_time,
                            end_time=end_time,
                            peak_time=pick_time,
                            peak_value=on_of[0],
                            phase='P')
                picks.append(pick)
        output.picks = picks
        return output

In [ ]:
# gdrive_path = '/content/drive/MyDrive'
# path = f'{gdrive_path}/Colab Notebooks/PhD-Thesis_ImanKahbasi'
# sys.path.append(path)
###
import MyFuncs as mf
from MyFuncs.structure import Result

In [ ]:
# gdrive_dir = '/content/drive/MyDrive'
# colab_dir = f'{gdrive_dir}/Colab Notebooks'
# working_dir = f'{colab_dir}/PhD-Thesis_ImanKahbasi'
working_dir = '.'
data_dir = f'{working_dir}/IIEES_2006-2023'

In [ ]:
fileObjPath = f'{data_dir}/eventid_waveformpath.pickle_OBJ'
eventid_waveformpath = {}

lst_waveform_dirs = glob.glob(f'{data_dir}/Waveforms/*')
for indx, path in enumerate(lst_waveform_dirs):
    eventid = os.path.basename(path).replace('waveform', '')
    path_waveform = mf.funcs.get_iiees_stream_path(path)
    # print(path_waveforms)
    # print(indx, eventid, path_waveform)
    eventid_waveformpath[eventid] = path_waveform
output = open(fileObjPath, 'wb')
pickle.dump(eventid_waveformpath, output)
output.close()
################################################################################
# fileObjPath = f'{data_dir}/eventid_waveformpath.pickle_OBJ'
# with open(fileObjPath, 'rb') as pickle_file:
#     eventid_waveformpath0 = pickle.load(pickle_file)
# eventid_waveformpath = eventid_waveformpath0

In [ ]:
###
# eventid_waveformpath = {k: eventid_waveformpath0[k] for k in list(eventid_waveformpath0.keys())[:5]}
###
# for k, v in eventid_waveformpath.items():
#     print(k, v)

In [ ]:
# path = "/home/ekarkooti/Documents/JSEE/SeisBench-JSEE_1402-08-16/IIEES_2006-2023/BIN_2004_2024-Cleaned.out"
# f = open(path)
# cat = read_events(f)
# print(cat)
# # cat = read_events('/home/ekarkooti/Documents/JSEE/SeisBench-JSEE_1402-08-16/IIEES_2018-2023/2018-2023_origin.out')

In [ ]:
cat = read_events(f'{data_dir}/BIN_2004_2024-Cleaned.out')
Details_lst_filenames = glob.glob(f'{data_dir}/Details/*')


In [ ]:
eventid2catalog = mf.funcs.eventid2catalog(Details_lst_filenames, cat)

In [ ]:
len(eventid2catalog)

In [ ]:
time_name = utc().strftime('%Y%m%dT%H%M%S')
root_output = f'{working_dir}/DL-picks-results_{time_name}'
proccessing_mode = 'CPU'
DPDN = False # Deep-DeNoiser

In [ ]:
if DPDN:
    model_dnoise = sbm.DeepDenoiser.from_pretrained('original')
    if proccessing_mode == 'GPU':
        model_dnoise.cuda()

## Define models

### Download and cash all models

In [ ]:
# lst_models = ['BasicPhaseAE', 'PhaseNet', 'EQTransformer', 'GPD']

# for model_name in lst_models:
#     exec(f"model = sbm.{model_name}")
#     lst = model.list_pretrained()
#     for pretrained in lst:
#         print(model_name, pretrained)
#         exec(f"model2 = model.from_pretrained('{pretrained}')")
#         print('Done')

### Make a list of target models that we want to apply on data

In [ ]:
df_models = pd.read_csv('models.csv', comment='#')
df_models['model'] = None
for indx, row in df_models.iterrows():
    if row.picker_name == 'STALTA':
        df_models.at[indx, 'model'] = stalta()
    else:
        dlmodel = eval(f"sbm.{row.picker_name}.from_pretrained(\'{row.training_data}\')")
        df_models.at[indx, 'model'] = dlmodel
        if row.processor == 'GPU':
            row.model.cuda()

In [ ]:
df_models

## Define filters

In [ ]:
low_corner_filter = [None]
high_corner_filter = [None]

## Processing data

In [ ]:
os.makedirs(root_output, exist_ok=True)

In [ ]:
# eventid_waveformpath_2 = {}
# ii = 0
# for eventid, wavepath in eventid_waveformpath.items():
#     eventid_waveformpath_2[eventid] = wavepath
#     ii += 1
#     if ii == 5:
#         break

In [ ]:
meta_header = ['eventid', 'wavename',
               'execution_time', 'RunTimeEQ',
               'freqmin', 'freqmax', 'DPDN',
               'picker_name', 'training_data', 'detection_threshold']

with open(f'metadata-of-run_{time_name}.csv', 'a') as metaout:
    metaout.write(','.join(meta_header)+'\n')

ii = 0
ii_t = len(eventid_waveformpath)
for freqmin, freqmax in zip(low_corner_filter, high_corner_filter):
    for eventid, wavepath in eventid_waveformpath.items():
        percent=ii/ii_t*100; print(f'{eventid} ({percent:.2f}%)', end='/'); ii += 1
        # if ii == ii_t:
        #     break
        wavename = os.path.basename(wavepath)
        wavename = wavename.split('.')[0][:-1]
        #
        st = read(wavepath)
        st.merge(-1)
        st.detrend('constant')
        st.merge(fill_value=0)
        st.resample(100)
        for tr in st:
            tr.stats.network = 'BI'
            tr.stats.channel = tr.stats.channel.replace(' ', 'H')
        ### filtering
        if freqmin or freqmax:
            print('Bandpss Filter')
            st.filter('bandpass', freqmin=freqmin, freqmax=freqmax,
                      zerophase=True)
        if DPDN:
            # st.filter('highpass', freq=0.5, zerophase=True)
            st = model_dnoise.annotate(st)
        ### Getting event from catalog
        try:
            ev = eventid2catalog[eventid]
        except KeyError as error:
            logging.error(error)
            continue
        ###
        for indx, model in df_models.iterrows():
            sRunTimeEQ = dt.now()
            classified = model.model.classify(st)
            eRunTimeEQ = dt.now()
            RunTimeEQ = (eRunTimeEQ - sRunTimeEQ).total_seconds()
            ### write results in an output file
            execution_time = sRunTimeEQ.strftime('%Y%m%dT%H%M%S')
            outpath = f'{root_output}/eID-{eventid}_wID-{wavename}_{model.picker_name}_{execution_time}.pickle'
            with open(outpath, 'wb') as outfile:
                pickle.dump(classified.picks, outfile)
            ###
            picker_name = model.picker_name
            training_data = model.training_data
            # detection_threshold = model.model.
            with open(f'metadata-of-run_{time_name}.csv', 'a') as metaout:
                # meta = [str(eval(_)) for _ in meta_header]
                meta = []
                for _ in meta_header:
                    try:
                        meta.append(str(eval(_)))
                    except:
                        meta.append('None')
                metaout.write(','.join(meta)+'\n')